In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import copy
import glob
import numpy as np
import toml
import sleap_io as sio
import re
import json
from tqdm.auto import tqdm
from sklearn.model_selection import ParameterGrid

In [3]:
# load in template

In [4]:
root_dir = "/storage/home/hcoda1/4/jmarkowitz30/shared_folder/active_lab_members/markowitz_jeffrey/active_projects/"

In [5]:
template_fname = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training/gridsearch_template/single_instance.json")
# template_fname = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training/gridsearch_template/bottomup_single_instance.json")
# template_fname = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training/gridsearch_template/topdown_centered_instance.json")
# MANUAL, all keypoints and kneejoints
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_manual_data_final_model")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_manual_data_subsample")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_manual_data_subsample_morerepeats")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_manual_data_parameter_sweep_part2")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_manual_data_different_modalities_kneejoints")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_manual_data_different_modalities_bottomup_v2")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_manual_data_parameter_sweep_topdown")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_manual_data_parameter_sweep_bottomup")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_manual_data_parameter_sweep_bottomup_parttwo")
save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_manual_data_subsample_reflectanceonly")

# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_manual_data_final_model_kneejoints")

# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_manual_data_subsample_fluo_aligned")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_manual_data_subsample_fluo_aligned_kneejoints")


# QDs, all keypoints
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_qds_heldout_camera_large_sets")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_qds_same_camera_v3")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_qds_heldout_camera_v3")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_qds_all_cameras_v3")

# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_qds_heldout_camera")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_qds_all_cameras")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_qds_all_cameras_large_models")


# QDs, kneejoints
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_qds_same_camera_kneejoints_v3")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_qds_heldout_camera_kneejoints_v3")
# save_dir = os.path.join(root_dir, "keypoints_basler_nir_plexiglass_arena/sleap_training_round2/keypoints_from_qds_all_cameras_kneejoints_v3")


os.makedirs(save_dir, exist_ok=False)
# save_dir = os.path.dirname(template_fname)

In [6]:
os.chdir(save_dir)

In [7]:
with open(template_fname, "r") as f:
    template_dct = json.load(f)

In [8]:
augment_keys = template_dct["optimization"]["augmentation_config"].keys()

In [9]:
for k, v in template_dct["model"]["backbone"].items():
    if v is not None:
        use_backbone = k
        break

for k, v in template_dct["model"]["heads"].items():
    if v is not None:
        use_head = k
        break

In [10]:
model_dct = template_dct["model"]["backbone"][use_backbone]
head_dct = template_dct["model"]["heads"][use_head]

In [15]:
if ("kneejoints" in save_dir) and ("from_qds" in save_dir):
    is_qd = True
    # can fit this all in memory, don't need the embedded data
    # training_data_lst = ["../_labels_qd/training_data_qdots_round2_kneejoints_embed-False.slp"]
    training_data_lst = ["../_labels_qd/kpoint_training_dataset_qd_alignment_kneejoints_reflect_only_round2_manual_labeling_nofluo_version-v1-embed-False.slp"]
elif ("from_qds" in save_dir):
    is_qd = True
    # training_data_lst = ["../_labels_qd/training_data_qdots_round2_embed-True-sleap-version.slp"]
    # training_data_lst = ["../_labels_qd/kpoint_training_dataset_qd_alignment_round2_manual_labeling_nofluo_v2-embed-False.slp"]
    # training_data_lst = ["../_labels_qd/kpoint_training_dataset_qd_alignment_round2_manual_labeling_nofluo_version-v1-embed-False.slp"]
    training_data_lst = ["../_labels_qd/kpoint_training_dataset_qd_alignment_round2_manual_labeling_nofluo_version-v1-embed-True-sleap-version.slp"]
elif ("kneejoints" in save_dir) and ("modalities" in save_dir) and ("fluo_aligned" in save_dir):
    is_qd = False
    nrepeats = 5
    training_data_lst = sorted(glob.glob("../_labels/basler-nir-plexiglass-arena-keypoints-fused-kneejoints*fluo-aligned.slp"))
elif ("kneejoints" in save_dir) and ("modalities" in save_dir) and ("fluo_aligned" not in save_dir):
    is_qd = False
    nrepeats = 5
    training_data_lst = sorted(glob.glob("../_labels/basler-nir-plexiglass-arena-keypoints-fused-kneejoints*None.slp"))
    training_data_lst = [_file for _file in training_data_lst if "fluo-aligned" not in _file]
elif ("modalities" in save_dir) and ("fluo_aligned" in save_dir):
    is_qd = False
    nrepeats = 5
    training_data_lst = sorted(glob.glob("../_labels/basler-nir-plexiglass-arena-keypoints-fused-round2*fluo-aligned.slp"))
elif ("modalities" in save_dir) and ("fluo_aligned" not in save_dir):  
    is_qd = False
    nrepeats = 5
    training_data_lst = sorted(glob.glob("../_labels/basler-nir-plexiglass-arena-keypoints-fused-round2*None.slp"))
    training_data_lst = [_file for _file in training_data_lst if "fluo-aligned" not in _file]
elif ("fluo_aligned" in save_dir):
    is_qd = False
    nrepeats = 2
    training_data_lst = sorted(glob.glob("../_labels/basler-nir-plexiglass-arena-keypoints-fused-round2_weights-None*fluo-aligned.slp"))
elif ("kneejoints" in save_dir):
    is_qd = False
    nrepeats = 2
    training_data_lst = sorted(glob.glob("../_labels/basler-nir-plexiglass-arena-keypoints-fused-kneejoints*None.slp"))    
    training_data_lst = [_file for _file in training_data_lst if ("fluo-aligned" not in _file) and ("(1.0, 0.0)" not in _file)]
elif ("subsample" in save_dir) and ("reflectanceonly" in save_dir):
    is_qd = False
    nrepeats = 7
    training_data_lst = sorted(glob.glob("../_labels/basler-nir-plexiglass-arena-keypoints-fused-round2_weights-(0.0, 1.0)*None.slp"))    
    training_data_lst = [_file for _file in training_data_lst if "fluo-aligned" not in _file]
elif ("subsample" in save_dir):
    is_qd = False
    nrepeats = 7
    training_data_lst = sorted(glob.glob("../_labels/basler-nir-plexiglass-arena-keypoints-fused-round2_weights-None*None.slp"))    
    training_data_lst = [_file for _file in training_data_lst if "fluo-aligned" not in _file]
else:
    is_qd = False
    nrepeats = 2
    training_data_lst = sorted(glob.glob("../_labels/basler-nir-plexiglass-arena-keypoints-fused-round2_weights-None*None.slp"))    
    training_data_lst = [_file for _file in training_data_lst if "fluo-aligned" not in _file]

In [16]:
if "same_camera" in save_dir:
    nrepeats = 1
    camera_type = "same"
elif "heldout_camera" in save_dir:
    nrepeats = 1
    camera_type = "heldout"
elif is_qd:
    nrepeats = 3
    camera_type = "all"
else:
    camera_type = "all"

In [17]:
preload_threshold = int(3e4)  # TYPICALLY 3e4, testing this out...
fix_inds = True  # # fix indices of train/va/test datasets...
fix_seed = True  # use same frames for each run/repeat
if is_qd and (camera_type == "heldout"):
    holdout_fraction = 0.05
    test_fraction = 0.05
elif is_qd and (camera_type == "same"):
    holdout_fraction = 0.05
    test_fraction = 0.05
elif is_qd and (camera_type == "all") and ("kneejoints" in save_dir):
    holdout_fraction = 0.05
    test_fraction = 0.05
elif is_qd and (camera_type == "all"):
    holdout_fraction = 0.025
    test_fraction = 0.025
elif "final_model" in save_dir:
    nrepeats = 5
    holdout_fraction = 0.05
    test_fraction = 0
else:
    holdout_fraction = 0.1
    test_fraction = 0.1

In [18]:
print(f"Training data: {training_data_lst}")
print(f"N(repeats): {nrepeats}")
print(f"Camera type: {camera_type}")
print(f"Is QD data: {is_qd}")
print(f"Validate fraction: {holdout_fraction}")
print(f"Test fraction: {test_fraction}")

Training data: ['../_labels/basler-nir-plexiglass-arena-keypoints-fused-round2_weights-(0.0, 1.0)_bpass-None.slp']
N(repeats): 7
Camera type: all
Is QD data: False
Validate fraction: 0.1
Test fraction: 0.1


In [20]:
template_dct["optimization"]["learning_rate_schedule"]["plateau_patience"] = 5
template_dct["optimization"]["early_stopping"]["plateau_patience"] = 10
template_dct["optimization"]["batch_size"] = 4
template_dct["optimization"][
    "min_batches_per_epoch"
] = 200  # 800ish seems to be a hard limit in terms of performance...
template_dct["optimization"]["epochs"] = 100 if "final_model" not in save_dir else 200  # NORMALLY 100
template_dct["data"]["labels"]["validation_fraction"] = holdout_fraction
template_dct["data"]["preprocessing"]["input_scaling"] = 1.0

# UNET
if use_backbone == "unet":
    # model_dct["filters"] = 64
    model_dct["filters"] = 64
    model_dct["max_stride"] = 64
    model_dct["stem_stride"] = None
    # model_dct["filters_rate"] = 1.25
    model_dct["filters_rate"] = 1.5
    model_dct["output_stride"] = 4
    model_dct["middle_block"] = True
    model_dct["up_interpolate"] = True
elif use_backbone == "resnet":
    template_dct["data"]["preprocessing"]["target_height"] = 1024
    template_dct["data"]["preprocessing"]["target_width"] = 1024
    model_dct["upsampling"]["skip_connections"] = "concatenate"
    model_dct["upsampling"]["transposed_conv_kernel_size"] = 8
    model_dct["weights"] = "tunable"
    model_dct["upsampling"]["batch_norm"] = True
    model_dct["upsampling"]["filters"] = 64
    model_dct["version"] = "ResNet152"
else:
    raise RuntimeError("Did not understand backbone")

if use_head == "single_instance":
    head_dct["sigma"] = 2.5
    head_dct["output_stride"] = 4

template_dct["outputs"]["delete_viz_images"] = True
template_dct["outputs"]["runs_folder"] = "models"
template_dct["optimization"]["augmentation_config"]["rotate"] = True
template_dct["optimization"]["augmentation_config"]["translate"] = True
template_dct["optimization"]["augmentation_config"]["scale"] = True
template_dct["optimization"]["augmentation_config"]["gaussian_noise"] = True
template_dct["optimization"]["augmentation_config"]["brightness"] = False
template_dct["optimization"]["augmentation_config"]["contrast"] = True
template_dct["optimization"]["augmentation_config"]["gaussian_noise_mean"] = 5.0
template_dct["optimization"]["augmentation_config"]["gaussian_noise_stddev"] = 1.0
template_dct["optimization"]["augmentation_config"]["translate_min"] = -50
template_dct["optimization"]["augmentation_config"]["translate_max"] = +50
template_dct["optimization"]["augmentation_config"]["scale_min"] = 0.85
template_dct["optimization"]["augmentation_config"]["scale_max"] = 1.15
template_dct["optimization"]["augmentation_config"]["rotation_min_angle"] = -15
template_dct["optimization"]["augmentation_config"]["rotation_max_angle"] = +15
template_dct["optimization"]["augmentation_config"]["brightness_min_val"] = -20
template_dct["optimization"]["augmentation_config"]["brightness_max_val"] = +20
template_dct["optimization"]["augmentation_config"]["contrast_min_gamma"] = 0.5
template_dct["optimization"]["augmentation_config"]["contrast_max_gamma"] = 2.0
template_dct["optimization"]["augmentation_config"][
    "random_flip"
] = True  # with symmetries saved good to go here...NORMALLY TRUE
template_dct["optimization"]["augmentation_config"]["flip_horizontal"] = True  # L/R flip (not up/down)...NORMALLY TRUE

In [41]:
param_types = {
    "data_labels": list(template_dct["data"]["labels"].keys()) + ["training_subsample"],
    "backbone": list(model_dct.keys()),
    "upsampling": [],
    "head": list(head_dct.keys()),
    "optimization": list(template_dct["optimization"].keys()),
    "augment": list(template_dct["optimization"]["augmentation_config"].keys()),
    "preprocessing": list(template_dct["data"]["preprocessing"].keys()),
    "hard_keypoint_mining": list(template_dct["optimization"]["hard_keypoint_mining"].keys()),
    "confmaps": [],
    "pafs": [],
}
if "upsampling" in model_dct.keys():
    param_types["upsampling"] = list(model_dct["upsampling"].keys())
if template_dct["model"]["heads"]["multi_instance"] is not None:
    param_types["confmaps"] = list(template_dct["model"]["heads"]["multi_instance"]["confmaps"].keys())
    param_types["pafs"] = list(template_dct["model"]["heads"]["multi_instance"]["pafs"].keys())


In [24]:
# label_range = [  60, 125, 250, 500, 1000, 2500, 5000, 10000, 25000, 50000, 100000 ]
if ("labels_qd" in training_data_lst[0]) and (camera_type == "same"):
    training_subsample = [  60, 125, 250, 500, 1000, 2500, 5000, 10000, 15000, 20000, None] # same camera
elif ("labels_qd" in training_data_lst[0]) and ((camera_type == "heldout") or (camera_type == "all")):
    training_subsample = [  60, 125, 250, 500, 1000, 2500, 5000, 10000, 15000, 25000, 50000, 75000, None]
else:
    training_subsample = [None]

In [27]:
# NEED A CUSTOM SKELETON FOR BOTTOMUP
force = True
import shutil

if "bottomup" in save_dir:

    new_edges = [
        (5, 4),  # back_middle → back_top
        (5, 6),  # back_middle → back_bottom
        (6, 2),  # back_bottom → hindleg_L
        (6, 3),  # back_bottom → hindleg_R
        (4, 0),  # back_top → foreleg_L
        (4, 1),  # back_top → foreleg_R
        (6, 7),  # back_bottom → tail_base
        (7, 8),  # tail_base → tail_middle
        (8, 9),  # tail_middle → tail_tip
    ]
    # preserve old nodes and symmetries, but need shallow tree structure for bottom up
    # new custom skeleton filenames, if they don't exist write them and remap training_data_lst
    new_training_data_lst = []
    for _fname in tqdm(training_data_lst):
        old_fname = os.path.splitext(_fname)[0]
        new_fname = f"{old_fname}_custom-skeleton.slp"
        if not os.path.exists(new_fname) or force:
            old_labels = sio.load_file(_fname)
            old_metadata_fname = _fname.replace(".slp", ".toml")
            old_skeleton = old_labels.skeleton

            new_skeleton = sio.Skeleton(name=old_skeleton.name)
            [new_skeleton.add_node(_node.name) for _node in old_skeleton.nodes]
            [new_skeleton.add_edge(_edge) for _edge in new_edges]
            [new_skeleton.add_symmetry(*_ind) for _ind in old_skeleton.symmetry_names]

            new_labeled_frames = []
            for lf in old_labels.labeled_frames:
                use_points = lf.instances[0].points
                use_points = {k.name: v for k, v in use_points.items()}
                instance = sio.Instance(skeleton=new_skeleton, points=use_points)
                labeled_frame = sio.LabeledFrame(video=lf.video, instances=[instance], frame_idx=lf.frame_idx)
                new_labeled_frames.append(labeled_frame)

            labels = sio.Labels(new_labeled_frames)
            labels.save(new_fname)
            new_metadata_fname = new_fname.replace(".slp", ".toml")
            shutil.copyfile(old_metadata_fname, new_metadata_fname)

        new_training_data_lst.append(new_fname)
        training_data_lst = new_training_data_lst

else:
    custom_skeleton = None

In [44]:
if "topdown" in save_dir:
    template_dct["data"]["instance_cropping"]["crop_size"] = 256

In [65]:
gridsearch_dct = {
    # DATA
    "training_labels": training_data_lst,
    # "training_subsample": [None],
    
    # RESNET
    # "version": ["ResNet152"],
    # "weights": ["tunable"],
    # "skip_connections": ["concatenate"],
    # "block_stride": [2, 4],
    "filters": [64.],
    "filters_rate": [2.],
    # "refine_convs": [2],
    # "transposed_conv_kernel_size": [8, 16],
    "max_stride": [64],
    # "filters": [64, 128, 256],
    # "filters_rate": [2],
    # "output_stride": [1],
    # "batch_size": [1, 2, 4, 8],
    # "batch_norm": [True],
    
    # UNET
    # "max_stride": [16, 32, 64],
    # "filters": [16, 32, 64],
    # "max_stride": [32, 64, 96],
    # "filters": [64],
    # "max_stride": [64],
    # "filters": [64, 96, 128],
    # "max_stride": [64, 96],
    # "output_stride": [1, 2, 4],
    # "up_interpolate": [True],
    # "middle_block": [True],
    # "output_stride": [4, 8, 16, 32],
    # "output_stride": [2, 4, 8],
    # "stem_stride": [None],
    # "filters_rate": [1.5],
    # "filters_rate": [1.5, 2.],
    # "filters_rate": [2],
    # BOTTOMUP
    # "sigma_pafs": [30, 40, 50, 60, 70, 80],
    # "output_stride_pafs": [4],
    # "sigma_confmaps": [2.5, 3.5],
    
    # OPTIMIZATION
    # "batch_size": [4], # ADD BATCH SIZE 1!
    # "scale": [.3],
    # "min_batches_per_epoch": [800],
    # "filters": [64, 96],
    # "online_mining": [False, True],
    # "initial_learning_rate": [1e-6, 2e-6, 3e-6],
    # "optimizer": ["adam"],
    # "min_batches_per_epoch": [200, 400],
    
    # PREPROCESSING
    # "ensure_rgb": [True],
    # "imagenet_mode": [None, "tf"],
    # "output_stride_backbone": [4, 8, 12],
    # "input_scaling": [1],
    
    # PRETRAINED
    # "pretrained": [True],
    # "encoder": ["resnet101", "densenet121"],
    # "decoder_batchnorm": [False, True],
    # "filters": [128, 256],
    # "filter_increase": [128, 256],
    # "max_stride": [32, 64],
    
    # HEAD
    # "sigma": [1., 1.5, 2., 2.5, 3., 3.5, 4.], # param sweep
    # "output_stride_head": [2., 4., 8.], # param sweep
    "sigma": [2.5], # optimal
    "output_stride_head": [4.], # optimal rounds 1/2
    # "sigma": [0.5], # check part 2
    
    # AUGMENTATION
    # "translate": [10, 50],
    # "scale": [.05, .15, .3],
    # "rotate": [5, 15],
    # "brightness": [5, 20],
    # "gaussian_noise_mean": [2.5, 5., 10.],
    # "gaussian_noise_stddev": [2.5, 5., 10.],
    # "contrast_min_gamma": [.5, .75],
    # "contrast_max_gamma": [1.5, 2.],
    # "brightness": [10., 20.],
}

In [66]:
if "manual_data" in save_dir and "bottomup" in save_dir:
    gridsearch_dct["filters_rate"] = [1.5] # typically 2
elif "manual_data" in save_dir:
    gridsearch_dct["filters_rate"] = [2.] # typically 2

if "subsample" in save_dir:
    gridsearch_dct["training_subsample"] = [50, 100, 200, 300, 400, 500, 600, None] # used this to address reviewer 3
else:
    gridsearch_dct["training_subsample"] = training_subsample

In [67]:
import tomli

In [68]:
metadata_fname = os.path.splitext(training_data_lst[0])[0] + ".toml"
with open(metadata_fname, "rb") as f:
    metadata = tomli.load(f)
try:
    metadata = metadata["segments_metadata"]
except KeyError:
    metadata = metadata["frames"]
cameras = sorted(list(set([_["camera"] for _ in metadata])))

In [69]:
if camera_type != "all":
    gridsearch_dct["camera"] = cameras

In [70]:
param_grid = ParameterGrid(gridsearch_dct)

In [71]:
re_camera = re.compile(r"(Basler\-[0-9|A-Z]+\-[0-9|A-Z]+)")

In [72]:
rng = np.random.default_rng()

In [73]:
test = list(param_grid)

In [74]:
import warnings

In [75]:
for _params in tqdm(param_grid):
    rng = np.random.default_rng(seed=0)
    subsample_rng = np.random.default_rng(seed=100)
    cur_camera = _params.pop("camera", None)

    try:
        metadata_fname = os.path.splitext(_params["training_labels"])[0] + ".toml"
        with open(metadata_fname, "rb") as f:
            metadata = tomli.load(f)
        try:
            metadata = metadata["segments_metadata"]
        except KeyError:
            metadata = metadata["frames"]
        sample_camera = [_["camera"] for _ in metadata]
    except FileNotFoundError:
        sleap_obj = sio.load_file(_params["training_labels"])
        sample_camera = [re_camera.search(_.video.filename).group(1) for _ in sleap_obj.labeled_frames]

    for _repeat in range(nrepeats):
        _iter_params = copy.deepcopy(_params)
        new_params = copy.deepcopy(template_dct)

        model_dct = new_params["model"]["backbone"][use_backbone]

        # new_params["data"]["preprocessing"]["input_scaling"] = _params["input_scaling"]
        for k, v in _iter_params.items():
            if (k in param_types["backbone"]) or k.endswith("backbone"):
                model_dct[k.replace("_backbone", "")] = v
            elif (k in param_types["data_labels"]) or k.endswith("data_labels"):
                new_params["data"]["labels"][k.replace("_data_labels", "")] = v
            elif (k in param_types["upsampling"]) or k.endswith("upsampling"):
                model_dct["upsampling"][k.replace("_upsampling", "")] = v
            elif (k in param_types["optimization"]) or k.endswith("optimization"):
                new_params["optimization"][k.replace("_optimization", "")] = v
            elif (k in param_types["hard_keypoint_mining"]) or k.endswith("hard_keypoint_mining"):
                new_params["optimization"]["hard_keypoint_mining"][k.replace("_hard_keypoint_mining", "")] = v
            elif (k in param_types["preprocessing"]) or k.endswith("preprocessing"):
                new_params["data"]["preprocessing"][k.replace("_preprocessing", "")] = v
            elif (k in param_types["head"]) or k.endswith("head"):
                new_params["model"]["heads"]["single_instance"][k.replace("_head", "")] = v
            elif (k in param_types["confmaps"]) or k.endswith("confmaps"):
                new_params["model"]["heads"]["multi_instance"]["confmaps"][k.replace("_confmaps", "")] = v
            elif (k in param_types["pafs"]) or k.endswith("pafs"):
                new_params["model"]["heads"]["multi_instance"]["pafs"][k.replace("_pafs", "")] = v
            elif (k in param_types["augment"]) or k.endswith("augment"):
                use_k = k.replace("_augment", "")
                try:
                    match_min = [_ for _ in augment_keys if f"{use_k}_min" in _][0]
                    match_max = [_ for _ in augment_keys if f"{use_k}_max" in _][0]
                    if use_k == "scale":
                        origin = 1
                    else:
                        origin = 0
                    new_params["optimization"]["augmentation_config"][match_min] = origin - v
                    new_params["optimization"]["augmentation_config"][match_max] = origin + v
                except IndexError:
                    new_params["optimization"]["augmentation_config"][use_k] = v
            else:
                raise RuntimeError(f"Did not understand {k}")

        # make interlaced lists...
        # labels = sio.load_slp(new_params["data"]["labels"]["training_labels"])
        training_subsample = new_params["data"]["labels"].pop("training_subsample", None)
        # nlabels = len(labels)

        if fix_inds:
            if fix_seed:
                rng = np.random.default_rng(seed=0)
    
            # HERE same/heldout/etc.
            if camera_type == "all":
                cameras = sorted(list(set(sample_camera)))
            elif camera_type == "heldout":
                cameras = sorted(list(set(sample_camera)))
            elif camera_type == "same":
                cameras = sorted(list(set(sample_camera)))
                cameras = [_cam for _cam in cameras if _cam == cur_camera]

            model_dct = new_params["model"]["backbone"][use_backbone]
            samples_by_camera = {}
            for _camera in cameras:
                samples_by_camera[_camera] = rng.permutation(
                    [i for i, camera in enumerate(sample_camera) if camera == _camera]
                ).tolist()

            if (camera_type == "all") or (camera_type == "same"):
                # can use same frame pool here...
                nlabels = sum([len(_) for _ in samples_by_camera.values()])

                # val_fraction = new_params["data"]["labels"]["validation_fraction"]
                train_fraction = 1 - holdout_fraction - test_fraction
                ntrain = np.floor(nlabels * train_fraction).astype("int")
                nholdout = np.floor(nlabels * holdout_fraction).astype("int")
                ntest = np.floor(nlabels * test_fraction).astype("int")

                all_idx = []
                stop = False
                # use same/heldout/etc here
                while not stop:
                    for _camera in cameras:
                        try:
                            all_idx.append(samples_by_camera[_camera].pop())
                        except IndexError:
                            continue
                    stop = all(
                        [len(v) == 0 for v in samples_by_camera.values()]
                    )  # stop once we've exhausted all frames...
                train_idx = all_idx[:ntrain]
                if (training_subsample is not None) and (training_subsample < len(train_idx)):
                    train_idx = subsample_rng.choice(train_idx, size=training_subsample).tolist()
                elif training_subsample is not None:
                    warnings.warn(
                        f"Training subsample exceeds availability: {cur_camera} nlabels {nlabels} subsample {training_subsample}"
                    )
                    continue
                validate_idx = all_idx[ntrain:ntrain + nholdout]
                test_idx = all_idx[ntrain + nholdout:ntrain + nholdout + ntest]
            else:
                # HELDOUT, train on OTHER cameras, test on THIS camera
                train_cameras = [_cam for _cam in cameras if _cam != cur_camera]
                ntrain_labels = sum([len(v) for k, v in samples_by_camera.items() if k in train_cameras])
                ntest_labels = len(samples_by_camera[cur_camera])
                ntotal_labels = ntrain_labels + ntest_labels

                all_train_idx = []
                stop = False
                # use same/heldout/etc here
                while not stop:
                    for _camera in train_cameras:
                        try:
                            all_train_idx.append(samples_by_camera[_camera].pop())
                        except IndexError:
                            continue
                    stop = all(
                        [len(v) == 0 for k, v in samples_by_camera.items() if k in train_cameras]
                    )  # stop once we've exhausted all frames...

                all_heldout_idx = samples_by_camera[cur_camera]
                nholdout = np.floor(ntest_labels * holdout_fraction).astype("int")
                ntest = np.floor(ntest_labels * test_fraction).astype("int")

                train_idx = all_train_idx
                if (training_subsample is not None) and (training_subsample < len(train_idx)):
                    train_idx = subsample_rng.choice(train_idx, size=training_subsample).tolist()
                elif training_subsample is not None:
                    warnings.warn(
                        f"Training subsample exceeds availability: {cur_camera} nlabels {ntrain_labels} subsample {training_subsample}"
                    )
                    continue
                validate_idx = all_heldout_idx[:nholdout]
                test_idx = all_heldout_idx[nholdout:nholdout + ntest]

            new_params["data"]["labels"]["training_inds"] = train_idx
            new_params["data"]["labels"]["validation_inds"] = validate_idx
            if len(test_idx) > 0:
                new_params["data"]["labels"]["test_inds"] = test_idx
            else:
                new_params["data"]["labels"]["test_inds"] = None
            new_params["data"]["labels"]["split_by_inds"] = True

        # nframes = len(sleap.load_file(new_params["data"]["labels"]["training_labels"]).labeled_frames)

        new_params["optimization"]["preload_data"] = (len(train_idx) < preload_threshold) and (
            len(validate_idx) < preload_threshold
        )

        new_name = "unet_"
        for k, v in _params.items():
            if k == "training_labels":
                use_v = os.path.basename(v)
                new_name += f"{k}-{use_v}_"
            else:
                new_name += f"{k}-{v}_"
        # new_name = new_name[:-1]
        new_name += f"repeat-{_repeat}"
        if cur_camera is not None:
            new_name += f"-{cur_camera}"
        print(f"N(train): {len(train_idx)}")
        print(f"N(test): {len(test_idx)}")
        print(f"N(validate): {len(validate_idx)}")

        new_params["outputs"]["run_name"] = new_name
        with open(os.path.join(save_dir, f"{new_name}.json"), "w") as f:
            json.dump(new_params, f, indent=4, sort_keys=False)

  0%|          | 0/8 [00:00<?, ?it/s]

N(train): 50
N(test): 86
N(validate): 86
N(train): 50
N(test): 86
N(validate): 86
N(train): 50
N(test): 86
N(validate): 86
N(train): 50
N(test): 86
N(validate): 86
N(train): 50
N(test): 86
N(validate): 86
N(train): 50
N(test): 86
N(validate): 86
N(train): 50
N(test): 86
N(validate): 86
N(train): 100
N(test): 86
N(validate): 86
N(train): 100
N(test): 86
N(validate): 86
N(train): 100
N(test): 86
N(validate): 86
N(train): 100
N(test): 86
N(validate): 86
N(train): 100
N(test): 86
N(validate): 86
N(train): 100
N(test): 86
N(validate): 86
N(train): 100
N(test): 86
N(validate): 86
N(train): 200
N(test): 86
N(validate): 86
N(train): 200
N(test): 86
N(validate): 86
N(train): 200
N(test): 86
N(validate): 86
N(train): 200
N(test): 86
N(validate): 86
N(train): 200
N(test): 86
N(validate): 86
N(train): 200
N(test): 86
N(validate): 86
N(train): 200
N(test): 86
N(validate): 86
N(train): 300
N(test): 86
N(validate): 86
N(train): 300
N(test): 86
N(validate): 86
N(train): 300
N(test): 86
N(validate): 86

In [76]:
new_params["model"]

{'backbone': {'leap': None,
  'unet': {'stem_stride': None,
   'max_stride': 64,
   'output_stride': 4,
   'filters': 64.0,
   'filters_rate': 2.0,
   'middle_block': True,
   'up_interpolate': True,
   'stacks': 1},
  'hourglass': None,
  'resnet': None,
  'pretrained_encoder': None},
 'heads': {'single_instance': {'part_names': None,
   'sigma': 2.5,
   'output_stride': 4.0,
   'loss_weight': 1.0,
   'offset_refinement': False},
  'centroid': None,
  'centered_instance': None,
  'multi_instance': None,
  'multi_class_bottomup': None,
  'multi_class_topdown': None},
 'base_checkpoint': None}

In [77]:
gridsearch_metadata = {
    "gridsearch_spec": gridsearch_dct,
    "nrepeats": nrepeats,
    "camera_type": camera_type,
    "is_qd": is_qd,
    "validate_fraction": holdout_fraction,
    "test_fraction": test_fraction,
    "template_fname": template_fname
}

In [78]:
with open(os.path.join(save_dir, "gridsearch_parameters.toml"), "w") as f:
    toml.dump(gridsearch_metadata, f)